In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\torre\AppData\Local\Temp\ipykernel_47684\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [4]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: 'data/kg_train.csv'

### Let's divide the training and test set into two partitions

In [ ]:
# Your code

## Data Preprocessing

In [ ]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\torre/nltk_data'
    - 'c:\\Users\\torre\\anaconda3\\nltk_data'
    - 'c:\\Users\\torre\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\torre\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\torre\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [ ]:
import re

def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [ ]:
import re

# Function to clean text based on the specified requirements
def clean_text_advanced(text):
    # Remove special characters (except spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove all single characters (like "a", "I" if they stand alone)
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    
    # Remove single characters from the start of words (if present)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove prefixed 'b' (common in byte representations)
    text = re.sub(r'\bb\b', '', text)
    
    # Convert text to lowercase
    text = text.lower()

    return text

# Apply cleaning function to training and test datasets
train_data['clean_text'] = train_data['text'].apply(clean_text_advanced)
test_data['clean_text'] = test_data['text'].apply(clean_text_advanced)

# Display cleaned data
train_data.head()


## Now let's work on removing stopwords
Remove the stopwords.

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Remove stopwords function
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

# Apply stopword removal
train_data['clean_text'] = train_data['clean_text'].apply(remove_stopwords)
test_data['clean_text'] = test_data['clean_text'].apply(remove_stopwords)

# Display cleaned text
train_data.head()


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [ ]:
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# Lemmatization function
def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)

# Apply lemmatization
train_data['clean_text'] = train_data['clean_text'].apply(lemmatize_text)
test_data['clean_text'] = test_data['clean_text'].apply(lemmatize_text)

# Display lemmatized text
train_data.head()


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
from collections import Counter

# Separate spam and ham messages
spam_words = " ".join(train_data[train_data['label'] == 1]['clean_text']).split()
ham_words = " ".join(train_data[train_data['label'] == 0]['clean_text']).split()

# Get top 10 words in each category
top_spam_words = Counter(spam_words).most_common(10)
top_ham_words = Counter(ham_words).most_common(10)

print("Top 10 Spam Words:", top_spam_words)
print("Top 10 Ham Words:", top_ham_words)


## Extra features

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer(max_features=5000)  # Use top 5000 words

# Fit and transform training data
X_train_bow = vectorizer.fit_transform(train_data['clean_text'])
X_test_bow = vectorizer.transform(test_data['clean_text'])

# Display the shape of the resulting matrices
print("Bag of Words (BoW) Train Shape:", X_train_bow.shape)
print("Bag of Words (BoW) Test Shape:", X_test_bow.shape)

# Display feature names (Top words used)
print("Top Words in BoW:", vectorizer.get_feature_names_out()[:10])


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Import required libraries
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Keep the top 5000 words

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['clean_text'])

# Transform the test data
X_test_tfidf = tfidf_vectorizer.transform(test_data['clean_text'])

# Print the shape of the vectorized dataset
print("TF-IDF Train Shape:", X_train_tfidf.shape)
print("TF-IDF Test Shape:", X_test_tfidf.shape)


## And the Train a Classifier?

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Multinomial Naive Bayes classifier
classifier = MultinomialNB()

# Train the classifier using TF-IDF transformed data
classifier.fit(X_train_tfidf, train_data['label'])

# Make predictions on the test set
y_pred = classifier.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(test_data['label'], y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

# Print classification report
print("Classification Report:\n", classification_report(test_data['label'], y_pred))


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Ensure test dataset contains labels (if missing, assume all are ham/spam for now)
if 'label' not in test_data.columns:
    print("'label' column is missing")
    test_data['label'] = 0  # Assign a default label (this should be corrected with actual labels)

# Feature Engineering - Trying different text representations
vectorizers = {
    "Bag of Words (BoW)": CountVectorizer(max_features=5000),
    "TF-IDF": TfidfVectorizer(max_features=5000)
}

best_accuracy = 0
best_vectorizer = None
best_X_train = None
best_X_test = None

# Train and evaluate using different text representations
for name, vectorizer in vectorizers.items():
    print(f"\nTraining with: {name}")

    # Transform text into feature vectors
    X_train = vectorizer.fit_transform(train_data['clean_text'])
    X_test = vectorizer.transform(test_data['clean_text'])

    # Train Naive Bayes Classifier (default MultinomialNB)
    classifier = MultinomialNB()
    classifier.fit(X_train, train_data['label'])

    # Predict on test set
    y_pred = classifier.predict(X_test)

    # Evaluate performance
    accuracy = accuracy_score(test_data['label'], y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")

    # Track best performance
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_vectorizer = name
        best_X_train = X_train
        best_X_test = X_test

# Display the best feature representation
print("\nBest Feature Representation:", best_vectorizer)
print("Best Accuracy:", best_accuracy)

# Final classification report for best approach
final_classifier = MultinomialNB()
final_classifier.fit(best_X_train, train_data['label'])
y_final_pred = final_classifier.predict(best_X_test)

print("\nFinal Classification Report:\n", classification_report(test_data['label'], y_final_pred))
